In [19]:
import pandas as pd
URM_path = "data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                sep=",",
                                # header=None,
                                dtype={0: int, 1: int, 2: int},
                                engine='python')

URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]
import scipy.sparse as sps

mapped_id, original_id = pd.factorize(URM_all_dataframe["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [20]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Hybrid import *
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Data_manager.UserUtils import *
import pandas as pd
import numpy as np


# Load the CSV file
file_path = 'updated_submission_ref.csv'
df = pd.read_csv(file_path)

URM_all = getURM_all()
users_few_iterations = URM_all.sum(axis=1)  
users_few_iterations = np.array(users_few_iterations).squeeze()
users_few_iterations = np.where((users_few_iterations < 2) * (users_few_iterations > 0))[0]

# slim = SLIMElasticNetRecommender(URM_all)
# slim.load_model(folder_path="slim_models/", file_name="slim_24")
# rp3 = RP3betaRecommender(URM_all)
# rp3.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
# itemknn = ItemKNNCFRecommender(URM_all)
# itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
#                                        feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
# 
# top = ScoresHybridRecommender(URM_all, slim, rp3, itemknn)
# top.fit(alpha=0.5072513954300486, beta=0.30902582824263064)

top = UserKNNCFRecommender(URM_all)
top.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)

item_index_to_original_id = item_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)

update_dict = {}
for user_id in users_few_iterations:
    recommendations = [item_index_to_original_id[index] for index in top.recommend(user_id, cutoff=10)]
    recommendations = " ".join(str(x) for x in recommendations)
    user_original = user_original_ID_to_index.reset_index().set_index(0).to_numpy().reshape(-1)[user_id]
    update_dict[user_original] = recommendations



Similarity column 12638 (100.0%), 8093.74 column/sec. Elapsed time 1.56 sec


In [21]:
len(update_dict)

948

In [16]:
update_dict

{9: '9018 2821 10108 17799 2282 20151 554 4535 115 2350',
 25: '6105 485 107 4089 4358 4066 5911 551 13332 18',
 27: '119 2472 4787 885 15850 1739 5507 237 1646 4907',
 31: '11954 322 1531 2088 3696 312 10209 10015 247 8395',
 37: '6108 6046 7635 16465 6315 516 19303 12141 3143 1953',
 53: '3970 859 6472 720 7075 3687 4819 6471 6273 9043',
 55: '5255 6617 13878 12798 6095 6941 7290 10578 6093 18909',
 66: '202 365 3290 642 1740 1114 2666 189 1782 3383',
 71: '738 4599 1387 1198 2490 5803 715 2370 2590 2127',
 80: '10305 4855 16542 4430 16599 653 20861 630 274 7746',
 87: '28 37 17 47 229 16 50 2 7 27',
 96: '72 766 1264 617 483 601 1346 401 256 845',
 105: '8752 19078 21 6820 18668 190 141 3 885 196',
 106: '1057 879 1885 7161 2051 2493 109 2592 9088 3674',
 116: '15 101 7 161 34 354 94 51 85 337',
 117: '88 72 339 217 1927 87 179 1010 183 538',
 120: '90 127 30 883 968 384 112 1154 20 1309',
 125: '15 101 7 161 34 354 94 51 85 337',
 148: '265 20 279 1750 1817 4344 116 7058 60 8596',


In [22]:
# Update the item_list for the specified userIDs
for user_id, new_item_list in update_dict.items():
    df.loc[df['user_id'] == user_id, 'item_list'] = new_item_list

# Save the updated dataframe back to a CSV file
df.to_csv('updated_submission.csv', index=False)